In [ ]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model, model_selection

In [ ]:
tweet = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [ ]:
print('There are {} rows and {} columns in train'.format(tweet.shape[0],tweet.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

In [ ]:
df=pd.concat([tweet,test])
tweet.shape
test.shape
df.shape

In [ ]:
#Removing the urls is a typical first step
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

In [ ]:
df['text']=df['text'].apply(lambda x : remove_URL(x))

In [ ]:
#Now to remove HTML tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [ ]:
df['text']=df['text'].apply(lambda x : remove_html(x))

In [ ]:
#Finally to remove emoji's
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
df['text']=df['text'].apply(lambda x: remove_emoji(x))

In [ ]:
#Also to remove punctuation
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [ ]:
df['text']=df['text'].apply(lambda x : remove_punct(x))

In [ ]:
training_df=df[df['target'].notnull()]
oot_df=df[df['target'].isnull()]

In [ ]:
training_df.shape
oot_df.shape

In [ ]:
oot_df = oot_df.drop('target', 1)

In [ ]:
oot_df.shape

In [ ]:
def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

list_corpus = training_df["text"].tolist()
list_labels = training_df["target"].tolist()
oot_corpus = oot_df["text"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=32)

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)
oot_df_counts = count_vectorizer.transform(oot_corpus)

In [ ]:
X_test_counts

In [ ]:
clf = linear_model.RidgeClassifier()

In [ ]:
scores = model_selection.cross_val_score(clf, X_train_counts, y_train, cv=3, scoring="f1")
scores

In [ ]:
scores = model_selection.cross_val_score(clf, X_test_counts, y_test, cv=3, scoring="f1")
scores

In [ ]:
clf.fit(X_train_counts, y_train)

In [ ]:
sample_submission = pd.read_csv("data/sample_submission.csv")

In [ ]:
sample_submission["target"] = clf.predict(oot_df_counts)

In [ ]:
sample_submission.describe()